In [1]:
# Importing necessary library
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pywt
import pandas as pd
import os
import time
from scipy import signal
from stingray import lightcurve
import sys
from stingray import Bispectrum
import warnings
import csv
warnings.filterwarnings('ignore')
%matplotlib inline

c:\Users\farra\anaconda3\envs\eeg_env\lib\site-packages\stingray\utils.py:54: UserWarning: Numba not installed. Faking it
  warnings.warn("Numba not installed. Faking it")


In [2]:
# file that will be skipped because of information loss
# zeros_test = ['co2a0000368_91.csv', 'co2c0000341_26.csv']
# zeros_train = ['co2a0000368_0.csv', 'co2a0000368_1.csv', 'co2a0000368_2.csv', 'co2a0000368_3.csv', 'co2a0000368_4.csv', 'co2a0000368_5.csv', 'co2c0000341_27.csv']

### Menghitung Matriks Cumulant orde ke-3

In [3]:
def calcCumulantOrde3(df_data, t, lag):
    # Compute the bispectrum of the signal
    lc = lightcurve.Lightcurve(t,df_data.T)
    bs = Bispectrum(lc, maxlag=lag)

    # Plot the bispectrum using contour plots
    # plt.contour(bs.freq, bs.freq, bs.bispec_mag)
    # plt.xlabel('f1')
    # plt.ylabel('f2')
    # plt.show()

    # Plot the bispectrum using mesh plots
    # fig = plt.figure()
    # ax = fig.add_subplot(111, projection='3d')
    # X, Y = np.meshgrid(bs.freq, bs.freq)
    # ax.plot_surface(X, Y, bs.bispec_mag)
    # ax.set_xlabel('f1')
    # ax.set_ylabel('f2')
    # ax.set_zlabel('Bispectrum')
    # plt.show()

    return bs.bispec_mag

### Creating Filter Matrix

In [4]:
def gkernel(k=2, l=1, sig=20):
    """
    Gaussian Kernel Creator via given length and sigma
    """

    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    bx = np.linspace(-(k - 1) / 2., (k - 1) / 2., k)

    xx, yy = np.meshgrid(ax, bx)

    kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sig))

    return kernel / np.max(kernel)

### Creating feature matrix

In [5]:
def create_feature(bispectrum, pyramid, fil_size):
    fsize = len(fil_size)
    feature_matrix = np.zeros([fsize,fsize])
    xtrack = 0
    ytrack = 0
    for xdim in range(fsize):
        for ydim in range(fsize):
            x,y = fil_size[xdim], fil_size[ydim]
            # print(xtrack, ytrack)
            
            feature_matrix[xdim][ydim] = np.mean(bispectrum[xtrack:xtrack+x, ytrack:ytrack+y] * pyramid[xtrack:xtrack+x, ytrack:ytrack+y])
            ytrack = ytrack+y
        ytrack = 0
        xtrack = xtrack+x
    final_feature = feature_matrix[np.triu(np.ones_like(feature_matrix, dtype=bool))]
    final_feature = final_feature[~np.isnan(final_feature)]

    return final_feature

    # x, y = np.meshgrid(np.arange(256), np.arange(256))
    # z = overall
    # z = z[:-1, :-1]
    # z_min, z_max = -np.abs(z).max(), np.abs(z).max()

    # plt.subplot()

    # plt.pcolormesh(x, y, z, 
    #             cmap =cm.coolwarm, 
    #             vmin = z_min, 
    #             vmax = z_max,
    #             edgecolors = 'face',
    #             shading ='flat')

    # plt.title('Filter')

    # # set the limits of the plot
    # # to the limits of the data
    # plt.axis([x.min(), x.max(), y.min(), y.max()])

    # plt.colorbar()
    # plt.show()

    # fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 5))
    # ax = plt.axes(projection="3d")
    # x2d, y2d = np.meshgrid(np.arange(256), np.arange(256))
    # ax.plot_surface(x2d, y2d, overall, cmap=cm.coolwarm,
    #                         linewidth=0, antialiased=False)
    # ax.set_xlabel('x')
    # ax.set_ylabel('y')
    # ax.set_zlabel('height')
    # plt.show()

### Persiapan data

In [6]:
# Define sampling frequency
fs = 256
t = np.arange(0, 1, 1/(fs*1))
t.shape

(256,)

In [7]:
def get_csv_EEG(filename):
    # Load data from CSV
    df_data = pd.read_csv(filename)
    return df_data, df_data.columns


### Perhitungan Feature Bispectrum

In [8]:
def create_pyramid(x, y):
    pyramid = np.zeros([x,y])
    x = pyramid.shape[0]
    y = pyramid.shape[1]

    for i in range(x // 2):
        for j in range(i, x - i):
            for h in range(i, y - i):
                pyramid[j, h] = i
    pyramid = pyramid/np.max(pyramid)
    return pyramid

In [9]:
def extract_feature(directory, lag, destination, filter_dim = 5, segment_time=1):
    fs = 256
    t = np.arange(0, 1, 1/(fs * segment_time))

    bis_dim = lag
    big_fil_size = bis_dim //2
    sigma = bis_dim//8
    if sigma == 0:
        sigma = 1
    fil_size = [big_fil_size // (2 ** i) for i in range(filter_dim)]
    
    if sum(fil_size) != bis_dim:
            fil_size = [value for value in fil_size if value != 0]
            if len(fil_size) == 1:
                fil_size.append(fil_size[0])
    
    fil_size[-1] = fil_size[-2] 
    pyramid = np.zeros([bis_dim,bis_dim])
    xtrack, ytrack = 0,0
    # print(fil_size)
    for xdim in range(len(fil_size)):
        for ydim in range(len(fil_size)):
            x,y = fil_size[xdim], fil_size[ydim]
            pyramid[xtrack:xtrack+x, ytrack:ytrack+y] = create_pyramid(x, y)
            ytrack = ytrack+y
        ytrack = 0
        xtrack = xtrack+x
    recap = pd.DataFrame(columns=['Wall Time', 'CPU Time'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            des_dir = os.path.join(destination +"_" + str(lag), foldername).lower()
            files = os.listdir(folder)
            for filename in files:
                cpu_start = time.process_time()
                wt_start = time.time()
                # if filename in zeros_train or filename in zeros_test:
                #     continue
                rel_path = os.path.join(directory, foldername, filename)
                # if 'metadata' in filename.lower():
                #     continue
                # trial_number = filename.split('.')[0].split('_')[1]
                des_file = foldername+'_'+ filename + '_bispectrum' +'.npy'
                if not os.path.exists(des_dir):
                    os.makedirs(des_dir)
                des_path = os.path.join(des_dir, des_file)
                if os.path.exists(des_path):
                    continue
                df_data, channel_name = get_csv_EEG(rel_path)
                bispectrum = []
                for channel in channel_name:
                    magnitude = calcCumulantOrde3(df_data[channel], t, lag)
                    magnitude = magnitude[:256, :256]
                    feature = create_feature(magnitude,pyramid,fil_size)
                    bispectrum.append(feature)
                bispectrum = np.array(bispectrum)
                bispectrum = bispectrum.flatten()
                np.save(des_path, bispectrum)
                wt_end = time.time()
                cpu_end = time.process_time()
                wall_time = wt_end - wt_start
                cpu_time = cpu_end - cpu_start
                recap_temp = pd.DataFrame([[wall_time, cpu_time]],columns=recap.columns)
                recap = pd.concat([recap, recap_temp], ignore_index=True)
                # pd.DataFrame(RWB.T).to_csv(des_path, index=False)
    recap_dir = os.path.join('./logs/Execution',directory.split('/')[1])
    if not os.path.exists(recap_dir):
        os.makedirs(recap_dir)
    recap_path = os.path.join(recap_dir,'recap_bispectrum_pyramid'+str(lag)+'.csv')
    recap.to_csv(recap_path)    


### Test

In [10]:
def extract_test(filename, lag, pyramid, fil_size):
    
    # if filename in zeros_train or filename in zeros_test:
    #     pass
    # if 'metadata' in filename.lower():
    #     pass
    
    df_data, channel_name = get_csv_EEG(filename)
    bispectrum = []
    for channel in channel_name:
        magnitude = calcCumulantOrde3(df_data[channel], t, lag)
        feature = create_feature(magnitude,pyramid,fil_size)
        bispectrum.append(feature)
    bispectrum = np.array(bispectrum)
    bispectrum = bispectrum.flatten()
    return bispectrum
                


In [11]:
bis_dim = 256
filter_dim = 5
big_fil_size = bis_dim //2
fil_size = [big_fil_size // (2 ** i) for i in range(filter_dim)]
fil_size[-1] = fil_size[-2] 
pyramid = np.zeros([bis_dim,bis_dim])
xtrack, ytrack = 0,0
for xdim in range(filter_dim):
    for ydim in range(filter_dim):
        x,y = fil_size[xdim], fil_size[ydim]
        pyramid[xtrack:xtrack+x, ytrack:ytrack+y] = create_pyramid(x,y)
        ytrack = ytrack+y
    ytrack = 0
    xtrack = xtrack+x
bs = extract_test('datasets/segmented_1 seconds/autism/Bader/segment_1001.csv', 256, pyramid, fil_size)
print(bs.shape)
print(bs)

(240,)
[5.94530780e+01 1.23083328e+02 3.20557708e+02 5.70623851e+02
 2.59943915e+03 2.86502477e+02 1.08109355e+03 2.45312488e+03
 1.36297327e+04 3.61290784e+03 9.65879821e+03 6.31180111e+04
 2.26235373e+04 1.33352870e+05 1.95587931e+06 1.74408434e+02
 2.49136782e+02 5.66659851e+02 8.77506391e+02 4.01519672e+03
 4.47685640e+02 1.40123001e+03 3.07072976e+03 1.83399514e+04
 4.90418013e+03 1.51129505e+04 8.77392161e+04 1.96558227e+04
 1.35885380e+05 1.63664402e+06 2.60336484e+02 2.70820731e+02
 4.58087968e+02 5.84523140e+02 4.44299260e+03 4.02709419e+02
 1.12948384e+03 1.78909605e+03 1.88091131e+04 2.46272345e+03
 6.15599609e+03 4.98042902e+04 5.24928817e+03 6.95046189e+04
 1.67991630e+06 5.83837170e+01 1.19743074e+02 2.13820464e+02
 3.22324431e+02 1.30720556e+03 4.73174869e+02 1.87562256e+03
 4.64337219e+03 2.24548571e+04 7.44518177e+03 9.71292961e+03
 5.00578604e+04 1.59996300e+04 6.77619982e+04 6.24976602e+05
 6.28467731e+00 2.83175710e+01 6.82361676e+01 9.08976280e+01
 5.89702742e+02 1

In [12]:
bis_dim = 256 #lag
filter_dim = 5

big_fil_size = bis_dim //2
sigma = bis_dim//8
if sigma == 0:
    sigma = 1
fil_size = [big_fil_size // (2 ** i) for i in range(filter_dim)]

if sum(fil_size) != bis_dim:
        fil_size = [value for value in fil_size if value != 0]
        if len(fil_size) == 1:
            fil_size.append(fil_size[0])

fil_size[-1] = fil_size[-2] 
pyramid = np.zeros([bis_dim,bis_dim])
xtrack, ytrack = 0,0
# print(fil_size)
for xdim in range(len(fil_size)):
    for ydim in range(len(fil_size)):
        x,y = fil_size[xdim], fil_size[ydim]
        pyramid[xtrack:xtrack+x, ytrack:ytrack+y] = gkernel(x, y, sigma)
        ytrack = ytrack+y
    ytrack = 0
    xtrack = xtrack+x

bs = extract_test('datasets/segmented_1 seconds/autism/Bader/segment_1001.csv', 256, pyramid, fil_size)
print(bs.shape)
print(bs)

(240,)
[6.38710041e+01 1.44136680e+02 4.01984283e+02 7.13410429e+02
 3.98650014e+03 5.95143717e+02 2.21081983e+03 4.92285206e+03
 3.32034370e+04 1.07812018e+04 2.58067154e+04 2.00754300e+05
 8.06595544e+04 6.07185909e+05 1.34583070e+07 1.88463157e+02
 2.91903695e+02 7.82031175e+02 1.09885873e+03 5.59342877e+03
 9.53668490e+02 3.22618684e+03 6.14905029e+03 4.12277116e+04
 1.75199800e+04 4.10619805e+04 2.48815435e+05 7.11887112e+04
 4.95139970e+05 8.27989994e+06 2.81364237e+02 3.25951615e+02
 6.90789341e+02 9.26830357e+02 6.45976477e+03 8.64419168e+02
 2.59760731e+03 4.35465997e+03 4.11110904e+04 1.02225643e+04
 2.13708930e+04 1.54788723e+05 2.99768064e+04 3.02667566e+05
 8.30012743e+06 6.31474336e+01 1.31135049e+02 2.56928905e+02
 3.30633221e+02 1.83187774e+03 8.96591138e+02 3.79812675e+03
 8.03126041e+03 5.45298137e+04 1.93836769e+04 2.23464419e+04
 1.57586521e+05 4.54446948e+04 2.65342485e+05 3.97650111e+06
 6.63482948e+00 3.08291214e+01 7.60339054e+01 1.03993357e+02
 6.40003322e+02 3

### Main Program

In [13]:
SEGMENT_TIME = 1

directory_segmented = f"datasets/segmented_{SEGMENT_TIME} seconds"
directory_feature = f"datasets/features/bispectrum_pyramid/segment_{SEGMENT_TIME} seconds"
directory_logs = f"logs/Execution/bispectrum_pyramid/segmented_{SEGMENT_TIME} seconds"

In [14]:
lags = [256]
for lag in lags:
    # extract_feature(os.path.join(directory_segmented, "autism"), lag, os.path.join(directory_feature, "autism"), segment_time=SEGMENT_TIME)
    extract_feature(os.path.join(directory_segmented, "normal"), lag, os.path.join(directory_feature, "normal"), segment_time=SEGMENT_TIME)

In [ ]:
# data = np.loadtxt('../out.csv', delimiter=",", skiprows=1, usecols=range(3,259))
# channel_name = np.loadtxt('../out.csv', delimiter=",", skiprows=1, usecols=1, dtype='str', encoding='utf-8')

# df_data = pd.DataFrame(data.T, columns=channel_name)
# df_data = df_data.drop(columns=['X', 'Y', 'nd'])
# bispectrum = []
# for channel in df_data.columns:
#     magnitude = calcCumulantOrde3(df_data[channel], t, 256)
#     bispectrum.append(magnitude)
# bispectrum = np.array(bispectrum)
# np.save('../bispectrum.npy', bispectrum)
# x = np.load('../bispectrum.npy')

# (bispectrum == x).all()